**DO NOT RUN EVERY BLOCK ONE BY ONE, EACH BLOCK IS UNIQUE AND DOES A SPECIFIC TASK, READ THROUGH THE TOP COMMENT OF EVERY BLOCK**

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#
#
# Move zip folder from drive to colab runtime
#
#

import zipfile
import shutil
import os

# Step 1: Copy the zip to Colab's local runtime
!cp /content/drive/MyDrive/diffpir_dark/Bright10k256.zip /content/
!cp /content/drive/MyDrive/diffpir_dark/Dark10k256.zip /content/

# Step 2: Unzip the dataset
!unzip -q /content/Bright10k256.zip -d /content/Bright10k256
!unzip -q /content/Dark10k256.zip -d /content/Dark10k256

In [ ]:
#
#
# Zip and send back to drive
#
#

import shutil

# Define your paths
source_folder = "/content/Dark10k256"  # Change this to your actual folder path
output_zip_path = "/content/Dark10k256.zip"
drive_dest = "/content/drive/MyDrive/diffpir_dark/Dark10k256.zip"

# 1. Zip the folder
shutil.make_archive(base_name=output_zip_path.replace('.zip', ''), format='zip', root_dir=source_folder)

# 2. Move the ZIP to your Google Drive
shutil.move(output_zip_path, drive_dest)

print(f"Zipped and moved to: {drive_dest}")

In [ ]:
#
#
# Flatten the directory - one folder with all images
#
#

flat_dir = "/content/Bright10k256_flat"
os.makedirs(flat_dir, exist_ok=True)

for root, dirs, files in os.walk("/content/Bright10k256"):
    for file in files:
        if file.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(flat_dir, file)
            shutil.copy2(src_path, dst_path)

print(f"Total images collected: {len(os.listdir(flat_dir))}")

In [ ]:
#
#
# Resize Images
#
#

import os
from PIL import Image

# Define source and target directories
src_dir = '/content/drive/MyDrive/diffpir_dark/69000_1'
dst_dir = '/content/drive/MyDrive/diffpir_dark/69_256'

# Create destination folder if it doesn't exist
os.makedirs(dst_dir, exist_ok=True)

# Loop through and resize
for filename in os.listdir(src_dir):
    if filename.lower().endswith('.png'):
        img_path = os.path.join(src_dir, filename)
        save_path = os.path.join(dst_dir, filename)

        # Open, resize, and save
        with Image.open(img_path) as img:
            img_resized = img.resize((256, 256), Image.LANCZOS)  # High-quality resampling
            img_resized.save(save_path)

print("Done resizing all images to 256x256.")

In [ ]:
#
#
# Darken Images
#
#

import os
import numpy as np
from PIL import Image, ImageEnhance
import random

def apply_darkening(image, seed=None):
    """
    Simulate realistic dark image from a bright one using multiple effects:
    - Brightness drop
    - Contrast reduction
    - Color desaturation
    - Noise addition
    - Gamma adjustment
    """
    if seed is not None:
        random.seed(seed)

    # 1. Random brightness reduction
    brightness_factor = random.uniform(0.4, 0.5)
    image = ImageEnhance.Brightness(image).enhance(brightness_factor)

    # 2. Reduce contrast slightly
    contrast_factor = random.uniform(0.7, 0.9)
    image = ImageEnhance.Contrast(image).enhance(contrast_factor)

    # 3. Slight desaturation
    color_factor = random.uniform(0.6, 0.9)
    image = ImageEnhance.Color(image).enhance(color_factor)

    # 4. Add realistic noise
    img_np = np.array(image).astype(np.float32)

    # Normalize to [0, 1]
    normalized = img_np / 255.0

    # Generate noise
    noise = np.random.normal(loc=0.0, scale=1.0, size=img_np.shape)

    # Scale noise inversely with brightness
    max_noise_strength = 10.0
    scaled_noise = noise * (1.0 - normalized) * max_noise_strength

    # Apply and clip
    img_np += scaled_noise
    img_np = np.clip(img_np, 0, 255).astype(np.uint8)

    # Convert back to image
    image = Image.fromarray(img_np)

    # 5. Gamma correction (simulate nonlinear light drop)
    gamma = random.uniform(1.7, 2.1)
    img_np = np.array(image).astype(np.float32) / 255.0
    img_np = np.power(img_np, gamma)
    img_np = (img_np * 255).clip(0, 255).astype(np.uint8)
    image = Image.fromarray(img_np)

    return image

bright_dir = '/content/Bright10k256/69000_256'
dark_dir = '/content/Dark10k256/69000_256'
os.makedirs(dark_dir, exist_ok=True)

# Apply enhanced darkening to each image
for i, filename in enumerate(os.listdir(bright_dir)):
    if filename.lower().endswith('.png'):
        path = os.path.join(bright_dir, filename)
        save_path = os.path.join(dark_dir, filename)

        with Image.open(path) as img:
            dark_img = apply_darkening(img, seed=i)  # seed for reproducibility
            dark_img.save(save_path)

print("Realistic darkening complete. Images saved.")

In [ ]:
#
#
# Convert JPG to PNG
#
#

from PIL import Image
import os

# Paths
input_dir = '/content/drive/MyDrive/diffpir_dark/BrightImages'
output_dir = '/content/drive/MyDrive/diffpir_dark/BrightImages'
os.makedirs(output_dir, exist_ok=True)

# Loop through all .jpg files and convert to .png
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.jpg') or filename.lower().endswith('.jpeg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path).convert('RGB')  # Ensure 3 channels
        new_filename = os.path.splitext(filename)[0] + '.png'
        img.save(os.path.join(output_dir, new_filename))

print("Conversion complete.")

In [ ]:
#
# CHANGE DIRECTORY
#
%cd /content/drive/MyDrive/diffpir_dark/DiffPIR

In [ ]:
# IF YOU GET ERROR ABOUT NOT FINDING THINGS CHANGE DIRECTORY WITH THE COMAND ABOVE
#
# CODE TO START TRAINING A NEW MODEL
#
# IT TAKES 2 FOLDERS, ONE WITH BRIGHT IMAGES, ONE WITH DARK IMAGES
# EACH IMAGE HAS A CORESPONDING NAMED PAIR

!python -m guided_diffusion.train_dark2bright \
    --data_dir_bright /content/Bright10k256_flat \
    --data_dir_dark /content/Dark10k256_flat \
    --image_size 256 \
    --num_channels 128 \
    --num_res_blocks 2 \
    --batch_size 8 \
    --image_cond True \
    --iterations 50000 \
    --log_interval 100 \
    --save_interval 10000 \
    --use_fp16 False

In [ ]:
# Inference

!python main_ddpir_brighten.py